In [ ]:
import json
import random

class Base:
    def __init__(self, name):
        self.name=name
        self.damage=100
        self.hp=100
        self.inventory=[]
    def apply_item(self, item): #this function adds the HP boost or Damage boost from the item in the JSON file
        if item["HP"]!=0:
            self.hp+=item["HP"]
            if item["HP"]>0:
                print(f"{self.name}'s HP boosted by {item['HP']}.")
            else:
                print(f"{self.name}'s HP decreased by {-item['HP']}.")
        if item["damage"]!=0:
            self.damage+=item["damage"]
            if item["damage"]>0:
                print(f"{self.name}'s damage boosted by {item['damage']}.")
            else:
                print(f"{self.name}'s damage decreased by {-item['damage']}.")
    def Heals (self, consumable_name):
        for item in self.inventory:
            if item["name"].lower()==consumable_name.lower() and item["Type"].lower()=="consumable":
                self.apply_item(item)
                self.inventory.remove(item)
                print(f"{self.name} used {item['name']}.")
                break
        else:
            print(f"'{consumable_name}' not found in inventory.")
    def equip_item(self, item_type, item_name):
        for item in self.inventory:
            if item["name"].lower()==item_name.lower() and item["Type"].lower()==item_type.lower():
                self.apply_item(item)
                self.inventory.remove(item)
                print(f"{self.name} equipped {item['name']}.")
                break
        else:
            print(f"{item_type}: '{item_name}' not found in inventory.")

    def access_inventory(self, for_healing=False):
        print("Inventory:")
        if for_healing:
            consumables=[item for item in self.inventory if item["Type"].lower()=="consumable"]
            for item in consumables:
                print(f"Name: {item['name']}")
                print(f"HP Boost: {item['HP']}")
                print(f"Description: {item['description']}")
        else:
            categories=["footwear", "leg armor", "chest plate", "weapon"]
            for category in categories:
                print(f"\n{category.capitalize()}:")
                category_items=[item for item in self.inventory if item["Type"].lower()==category.lower()]
                for item in category_items:
                    print(f"Name: {item['name']}, Type: {item['Type']}")
    def attributes(self):
        print(f"Ready for the BIG day {self.name}! Choose your armaments:")


class Character(Base):
    def __init__(self, name, inventory):
        super().__init__(name)
        self.inventory=inventory
        self.equipped_categories={"footwear": False, "leg armor": False, "chest plate": False, "headgear": False, "weapon": False} #Makes sure character doesnt start with any items equiped 
    def choose_attributes(self):
        super().attributes()
        self.hp+=10
    def choose_weapon(self, weapon_name):
        for item in self.inventory:
            if item["name"].lower()==weapon_name.lower() and item["Type"].lower()=="weapon":
                self.apply_item(item)
                self.inventory.remove(item)
                self.equipped_categories["weapon"]=True
                print(f"{self.name} equipped {item['name']}.")
                break
        else:
            print(f"Weapon '{weapon_name}' found in inventory.")
    def access_inventory(self, for_healing=False, during_battle=False):
        print("Inventory:")
        if for_healing:
            consumables=[item for item in self.inventory if item["Type"].lower()=="consumable"]
            for item in consumables:
                print(f"Name: {item['name']}")
                print(f"HP Boost: {item['HP']}")
                print(f"Description: {item['description']}")
                print()
        elif during_battle:
            categories=["footwear", "headgear", "chest plate", "leg armor", "weapon"]
            print("Available Categories:")
            print(", ".join(categories)) #Aided by partner to display categories better 
            selected_category=input("Select a category: ").lower()
            print(f"\n{selected_category.capitalize()}:")
            category_items=[item for item in self.inventory if item["Type"].lower() == selected_category]
            for item in category_items:
                print(f"Name: {item['name']}")
                print(f"Description: {item['description']}")
                if item["Type"].lower()=="weapon":
                    print(f"Damage Boost: {item['damage']}")
                print(f"HP Boost: {item['HP']}")
                print()
            return selected_category
        else:
            categories=["footwear", "headgear", "chest plate", "leg armor", "weapon"]
            for category in categories:
                print(f"\n{category.capitalize()}:")
                category_items=[item for item in self.inventory if item["Type"].lower()==category.lower()]
                for item in category_items:
                    print(f"Name: {item['name']}, Type: {item['Type']}")
    def equip_item(self, item_type, item_name): #User can equip an item from the given inventory
        while True:
            for item in self.inventory:
                if item["name"].lower()==item_name.lower() and item["Type"].lower()==item_type.lower():
                    self.apply_item(item)
                    self.inventory.remove(item)
                    self.equipped_categories[item_type.lower()] = True
                    print(f"{self.name} equipped {item['name']}.")
                    return  # Exits the loop after equipping an item
            print(f"{item_type} with name '{item_name}' not found in inventory. Please choose from the available items.")
            self.access_inventory()  # Shows available items in the inventory
            item_name=input(f"Enter the name of the {item_type} to equip: ")


def choose_character():
    print("Choose a character:")
    print("1. Jefe")
    print("2. Mafins")
    print("3. Manchis")
    print("4. Tipiti")
    print("5. Panzon")
    print("6. Lagartijo")
    choice=input("Enter the corresponding number: ")
    characters={
        "1": "Jefe",
        "2": "Mafins",
        "3": "Manchis",
        "4": "Tipiti",
        "5": "Panzon",
        "6": "Lagartijo" }
    if choice in characters:
        return Character(characters[choice], load_inventory())
    else:
        print("Invalid choice. Please enter a valid number.")
        return choose_character()
def load_inventory():
    with open("ItemValues.json", "r") as f:
        return json.load(f)["items"]
def show_items(items):
    for item in items:
        print(f"Name: {item['name']}")
        print(f"Description: {item['description']}")
        if item["Type"].lower() == "weapon":
            print(f"Damage Boost: {item['damage']}")
            print(f"HP Boost: {item['HP']}")
        else:
            print(f"HP Boost: {item['HP']}")
        print()
def battle(player, opponent):
    attack_messages=["Lolo Charged!", "Double Kick","Mega screech","Mike Tyson level hit from Lolo", "Body Slam"]
    print(f"{player.name} is battling {opponent.name}!")
    while player.hp>0 and opponent.hp>0:  # Fight until one of the characters' HP reaches zero, battle to the death
        print("\nNext Round:")
        print(f"Player HP: {player.hp}, Opponent HP: {opponent.hp}")
        if player.hp>0:  # Only allow actions if the player is still alive
            choice=input("\nDo you want to (i)inventory, (f)ight, or (c)consume using a consumable? (i/f/c): ")
            if choice.lower()=="i":
                selected_category=player.access_inventory(during_battle=True)  # Allow changing one item during battle, some issue when the inventory was opened
                change_item=input("Do you want to change an equipped item? (yes/no): ")
                if change_item.lower() in ["yes", "y"]:
                    item_name=input(f"Enter the name of the {selected_category} to equip: ")
                    player.equip_item(selected_category, item_name)
                continue  # Skip the attack phase, when inventory is accessed there are no attacks
            elif choice.lower()=="c":
                player.access_inventory(for_healing=True, during_battle=True)
                consumable_name=input("Enter the name of the consumable to use: ")
                player.Heals(consumable_name)
                print("\nUpdated character stats:")
                print(f"Player HP: {player.hp}, Enemy HP: {opponent.hp}")
                continue  # Skip the attack phase
            elif choice.lower()!="f":
                print("Invalid choice. Continuing to fight.")
                continue  # Skip the attack phase
        
        # Player attacks opponent
        player_attack=player.damage # Use damage value from the selected weapon, fixed the weapons doing random damage
        opponent.hp-=player_attack
        print(f"{player.name} attacks {opponent.name} for {player_attack} damage!")
        
        if opponent.hp<=0:
            print(f"{opponent.name} has been defeated! We have a new champion, but do they eat bunnies")
            break
      
        # Opponent attacks player
        opponent_attack=random.randint(50, 150)  # Random attack value for Lolo
        attack_messages=random.choice(attack_messages)
        player.hp-=opponent_attack
        print(f"{opponent.name} attacks {player.name} for {opponent_attack} damage!")
        
        # Check if player is defeated
        if player.hp<=0:
            print(f"{player.name} is defeated!!! Another win for Lolo the man eating bunny!")
            break

    else:
        print("The battle is over!")
        
    while True:
        play_again=input("Do you want to play again? (yes/no): ")
        if play_again.lower() in ["yes", "y"]:
            main()  # Restart the battle
            break
        elif play_again.lower() in ["no", "n"]:
            print("Wake up!!! why were you mumbling about some Lolo? Was it a dream? ")
            break
        else:
            print("Invalid input. Please enter 'yes' or 'no'.")


def main():
    selected_character=choose_character()
    print("Welcome to La Casa arena! Prepare yourself to be Lolo's next meal. You seem nice and healthy; you will be a great snack. We hope you entertain us!!")
    selected_character.choose_attributes()
    selected_items={}  # Dictionary to store selected items
    for category in selected_character.equipped_categories.keys():
        print(f"\nChoose {category}:")
        category_items=[item for item in selected_character.inventory if item["Type"].lower()==category]
        show_items(category_items)
        while not selected_character.equipped_categories[category]:
            item_name=input(f"Enter the name of the {category} to equip: ")
            selected_character.equip_item(category, item_name)
            selected_items[category]=item_name  # Store selected item
            if category.lower()!="weapon":  # Only prompt for the next category if not a weapon
                break  # Exit the loop after equipping an item
        print("=" * 50)  # Add a line of "=" to separate categories visually
    print("\nEquipped Items:")
    for category, item_name in selected_items.items():
        print(f"{category.capitalize()}: {item_name}")

    print("\nStarting the fight...\n")

    opponent=Character("Lolo the Man-Eating Bunny", load_inventory())

    opponent.hp=1000
    opponent.damage=200

    battle(selected_character, opponent)
    
if __name__=="__main__":
    main()
